# COVID-19 in the Czech Republic

In [50]:
# Imports
import geopandas as gpd
import pandas as pd
from math import ceil
import json
import os
from datetime import datetime
import requests

# Bokeh
from bokeh.io import output_notebook, show, output_file, export_png
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, LogColorMapper, ColorBar
from bokeh import palettes

# Image Processing
from PIL import Image, ImageDraw, ImageFont

In [51]:
# Sources
covid_data_source = "https://onemocneni-aktualne.mzcr.cz/api/v2/covid-19/kraj-okres-nakazeni-vyleceni-umrti.csv"
covidfile = './covid/kraj-okres-nakazeni-vyleceni-umrti.csv'

shapefile = './cz_shapefile/JTSK/SPH_OKRES.shp'

In [52]:
# Get fresh covid data if file more than 1 day old

def hours_since_modified(target):
    now = datetime.timestamp(datetime.now())
    target = os.stat(target).st_mtime
    diff_hours = ( now - target ) / 60 / 60
    return round(diff_hours)

def download_covid_data(url):
    r = requests.get(url)
    open(covidfile, 'w').write(r.text)

if not os.path.exists(covidfile) or hours_since_modified(covidfile) > 24:
    print('file is {} hours old. Updating file..'.format(hours_since_modifiedmodified(covidfile)), end="")
    download_covid_data(covid_data_source)
    size = round(os.path.getsize(covidfile) / (1024*1024), 2)
    print('Downloaded {} MB!'.format(size))

In [53]:
# Load into DataFrames

# Read data into Pandas
gdf = gpd.read_file(shapefile)[['KOD_LAU1','NAZEV_LAU1','geometry']]
df = pd.read_csv(covidfile)

# Simplify topology 
gdf['geometry'] = gdf['geometry'].simplify(300) # Probably need to preserve_topology=True

# Drop NaN values
df.dropna(inplace=True)

In [54]:
df

,datum,kraj_nuts_kod,okres_lau_kod,kumulativni_pocet_nakazenych,kumulativni_pocet_vylecenych,kumulativni_pocet_umrti
0,2020-03-01,CZ010,CZ0100,2,0,0
1,2020-03-01,CZ020,CZ020A,0,0,0
2,2020-03-01,CZ020,CZ020B,0,0,0
3,2020-03-01,CZ020,CZ020C,0,0,0
4,2020-03-01,CZ020,CZ0201,0,0,0
...,...,...,...,...,...,...
25656,2021-01-23,CZ080,CZ0802,20569,19154,387
25657,2021-01-23,CZ080,CZ0803,23246,21382,421
25658,2021-01-23,CZ080,CZ0804,13428,12362,177
25659,2021-01-23,CZ080,CZ0805,20491,18825,286


In [55]:
gdf

,KOD_LAU1,NAZEV_LAU1,geometry
0,CZ0100,Hlavní město Praha,"POLYGON ((-736538.020 -1053708.250, -737677.83..."
1,CZ0201,Benešov,"POLYGON ((-746500.570 -1072617.070, -746261.63..."
2,CZ0202,Beroun,"POLYGON ((-760901.670 -1049328.700, -761489.83..."
3,CZ0203,Kladno,"POLYGON ((-776276.650 -1024382.940, -776267.08..."
4,CZ0204,Kolín,"POLYGON ((-675684.020 -1058939.760, -677411.67..."
...,...,...,...
72,CZ0806,Ostrava-město,"POLYGON ((-468269.230 -1094196.750, -467350.78..."
73,CZ0724,Zlín,"POLYGON ((-511885.360 -1151869.790, -511474.07..."
74,CZ0721,Kroměříž,"POLYGON ((-559576.450 -1158750.950, -558171.22..."
75,CZ0722,Uherské Hradiště,"POLYGON ((-536685.800 -1172033.730, -535832.16..."


In [56]:
# Calculate and add columns with new cases per day and 7 day running sum of new cases

df_extended = pd.DataFrame()

for okres in df.okres_lau_kod.unique():
    okres_data = df.loc[ df.okres_lau_kod == okres].sort_values('datum')
    okres_data['new_cases'] =  okres_data.kumulativni_pocet_nakazenych - okres_data.kumulativni_pocet_nakazenych.shift(1)
    okres_data['week_total'] = okres_data['new_cases'].rolling(7).sum()
    
    df_extended = df_extended.append(okres_data)

df_extended.fillna(0, inplace=True)

max_cases = ceil( df_extended['week_total'].max() / 1000 ) * 1000

In [57]:
df_extended.loc[df_extended.datum == "2020-03-01"]

,datum,kraj_nuts_kod,okres_lau_kod,kumulativni_pocet_nakazenych,kumulativni_pocet_vylecenych,kumulativni_pocet_umrti,new_cases,week_total
0,2020-03-01,CZ010,CZ0100,2,0,0,0.0,0.0
1,2020-03-01,CZ020,CZ020A,0,0,0,0.0,0.0
2,2020-03-01,CZ020,CZ020B,0,0,0,0.0,0.0
3,2020-03-01,CZ020,CZ020C,0,0,0,0.0,0.0
4,2020-03-01,CZ020,CZ0201,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...
72,2020-03-01,CZ080,CZ0802,0,0,0,0.0,0.0
73,2020-03-01,CZ080,CZ0803,0,0,0,0.0,0.0
74,2020-03-01,CZ080,CZ0804,0,0,0,0.0,0.0
75,2020-03-01,CZ080,CZ0805,0,0,0,0.0,0.0


In [60]:
# Iterate over days and export visualisation to PNG
for date in df_extended.datum.unique()[:1]:
    
    # Limit to one day for now
    df_today = df_extended.loc[df_extended['datum'] == str(date)]
    
    # Merge datasets
    merged = gdf.merge(df_today, left_on='KOD_LAU1', right_on='okres_lau_kod')[
        ['okres_lau_kod','NAZEV_LAU1','datum','geometry','new_cases','week_total']
    ]
    
    merged_json = json.loads(merged.to_json())
    
    json_data = json.dumps(merged_json)

    # Export JSON to file if ya wanna
#     with open('merged_json.json', 'w') as f:
#         json.dump(merged_json, f)

    geosource = GeoJSONDataSource(geojson = json_data)

    # Color palette
    palette = palettes.Plasma256

    # Invert pallete so that highest number is darkest
    palette = palette[::-1]
    palette = tuple(list(['#e4e4e4']) + list(palette))

    # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = 0, high = max_cases)

    # Define custom tick labels for color bar.
#     tick_labels = {'0': '0', '10': '10'}

    #Create color bar. 
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 15,
    border_line_color='white',location = (0,50), orientation = 'horizontal')

    #Create figure object.
    p = figure(title = None, plot_height = 600 , plot_width = 950, toolbar_location = None)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    p.axis.visible = False

    #Add patch renderer to figure. 
    p.patches('xs','ys', 
              source = geosource,fill_color = {'field' :'week_total', 'transform' : color_mapper},
              line_color = 'white', line_width = 1, fill_alpha = 1)

    #Specify figure layout.
    p.add_layout(color_bar, 'below')
    
    # Export to PNG
    export_png(p, filename=f'./covid/frames/{date}.png')
    

    #Display figure inline in Jupyter Notebook.
    output_notebook()

    #Display figure.
    show(p)

Loading BokehJS ...

# Add Text and Export to GIF

In [90]:
# Iterate over exported PNGs, add text, combine to GIF

frames = []
ms_per_frame = 15
    
for date in df.datum.unique()[:5]:
    img = Image.open(f'./covid/frames/{date}.png')
    draw = ImageDraw.Draw(img)
    
    # Add large date title
    fnt = ImageFont.truetype('./covid/Ubuntu-Medium.ttf', size=50)
    draw.text((640, 40), date, fill="black", font=fnt)
    
    # Add title to legend
    fnt = ImageFont.truetype('./covid/Ubuntu-Medium.ttf', size=12)
    draw.text((30, 470), "CZECH REPUBLIC", fill="black", font=fnt)
    
    # Add title to legend
    fnt = ImageFont.truetype('./covid/Ubuntu-Medium.ttf', size=12)
    draw.text((30, 485), "NEW CASES OF COVID-19 IN LAST 7 DAYS", fill="black", font=fnt)

    frames.append(img)

for i in range(0, round(2000/ms_per_frame)):
    frames.append(frames[-1])
    
# gif_palette = ImagePalette.ImagePalette(mode='HEX', palette=list(palette[:-1]), size=256)
    
frames[0].save('./covid/export_v3_2021-02-01.gif', format='GIF',
               append_images=frames[1:], save_all=True, duration=ms_per_frame, 
               disposal=1)

In [72]:
from PIL import ImagePalette

In [74]:
help(ImagePalette.ImagePalette)

Help on class ImagePalette in module PIL.ImagePalette:

class ImagePalette(builtins.object)
 |  ImagePalette(mode='RGB', palette=None, size=0)
 |  
 |  Color palette for palette mapped images
 |  
 |  :param mode: The mode to use for the Palette. See:
 |      :ref:`concept-modes`. Defaults to "RGB"
 |  :param palette: An optional palette. If given, it must be a bytearray,
 |      an array or a list of ints between 0-255 and of length ``size``
 |      times the number of colors in ``mode``. The list must be aligned
 |      by channel (All R values must be contiguous in the list before G
 |      and B values.) Defaults to 0 through 255 per channel.
 |  :param size: An optional palette size. If given, it cannot be equal to
 |      or greater than 256. Defaults to 0.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, mode='RGB', palette=None, size=0)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  copy(self)
 |  
 |  getcolor(self, color)
 |      Given a

In [81]:
len(palette[:-1])

256